Princess Dickens, 
LING 6300, HW2

# Cornell Movie Review

1. There will be two classes: positive and negative. Build a count dictionary for each class.
key = word
value = count

Ex) postive('great') = 669

2. Figure out what the vocabulary is for both classes together - set() - and the size len(vocabulary). This way, you can initialize each dictionary with already-smoothed values, and keys from BOTH bags.
    - Get set of words for both bags
    - Initialize pos dict with all words and a count of 0.5
    - Initialize neg dict with all words and a count of 0.5
    - Go through training data and add real counts to each dict respectively
    

3. Design a function that takes a word and the dictionary and the vocabulary set (as a minimum) as arguments and the probability in that class.

Ex) getp(mypositivecounts, vocabulary, 'great')

4. Add smoothing to this function.

In [34]:
import os 
import math
from decimal import *
posDict = {}
negDict = {}
vocabulary = set()

# Read lines from each file and split into list of words
# Create dict {word:count}

lines = [line.strip() for line in open('pos.train')]
poswords = []
for l in lines:
     poswords.extend(l.split())

for word in poswords:
    if word not in posDict:
        posDict[word] = 1
    else:
        posDict[word] += 1

lines2 = [line.strip() for line in open("neg.train")]
negwords = []
for l in lines2:
    negwords.extend(l.split())

for word in negwords:
    if word not in negDict:
        negDict[word] = 1
    else:
        negDict[word] += 1

# test posDict['great']
print("posDict['great'] = ", posDict['great'])
# test negDict['great']
print("negDict['great'] = ", negDict['great'])
# size of posDict
print("Size of posDict: ", len(posDict))
# size of negDict
print("Size of negDict: ", len(negDict))
# total tokens in posDict
pos_token_count = sum(posDict.values())
print("Total tokens in posDict: ", pos_token_count)
# total tokens in negDict
neg_token_count = sum(negDict.values())
print("Total tokens in negDict: ", neg_token_count)

# merge two dicts to create one set "vocabulary"

for key in posDict:
    vocabulary.add(key)
for key in negDict:
    vocabulary.add(key)
    
print("Total types in vocabulary: ", len(vocabulary))

# Add other words to posDict and negDict with count = 0

for word in vocabulary:
    if word not in posDict:
        posDict[word] = 0
    if word not in negDict:
        negDict[word] = 0

# new size of posDict
print("New size of posDict: ", len(posDict))
# new size of negDict
print("New size of negDict: ", len(negDict))

# get total count and pass in as argument in getP (pos and neg separately)
#pos_token_count = sum(posDict.values())
#neg_token_count = sum(negDict.values())

def getP(inputDict, inputTokenCount, vocabulary, inputWord):
    # Add smoothing 
    SMOOTHING_VAR = 1
    count_with_smoothing = inputDict[inputWord] + SMOOTHING_VAR
    new_total_tokens = inputTokenCount + (len(vocabulary) * SMOOTHING_VAR)
    
    # prob = (count for this word + smoothing val) / (total tokens + smoothing for every TYPE)
    probability = count_with_smoothing / new_total_tokens       
    
    return round(probability, 6)

print("p('great'|pos) = ", getP(posDict, pos_token_count, vocabulary, 'great'))
print("p('great'|neg) = ", getP(negDict, neg_token_count, vocabulary, 'great'))

posDict['great'] =  669
negDict['great'] =  350
Size of posDict:  35093
Size of negDict:  32891
Total tokens in posDict:  710749
Total tokens in negDict:  635078
Total types in vocabulary:  48622
New size of posDict:  48622
New size of negDict:  48622
p('great'|pos) =  0.000882
p('great'|neg) =  0.000513


# Bag-of-words Naive Bayes Classifier
Now that I've got a way of calculating the probability that a WORD is either positive or negative, I can add up the probabilities of a set number of words in a movie review to predict whether a movie review is positive or negative.

Algorithm:
1. Find the probability of each word, take the LOG of it, and add its score to the total. Do this with the positive and negative dictionaries separately. 
2. Compare the total scores for pos and neg. Whichever number is higher is the winner.
3. Keep count of the winner to see how many positive and negative reviews it gets right.

In [35]:
def classify_review(listWords, vocabulary):
    #accepts list of words from review as argument instead of fileName
    pos_prior = 0.5
    posScore = math.log(pos_prior)
    for word in listWords:
        if word in vocabulary:
            posScore += math.log(getP(posDict, pos_token_count, vocabulary, word))
    
    neg_prior = 0.5
    negScore = math.log(neg_prior)
    for word in listWords:
        if word in vocabulary:
            negScore += math.log(getP(negDict, neg_token_count, vocabulary, word))
        
    if posScore > negScore:
        return 0
    else:
        return 1

    
# for POSITIVE test set

pos_review_count = 0
neg_review_count = 0

# read multiple text files in a single directory

file_dir1 = 'test/pos/'

for fileName in os.listdir(file_dir1):
    wordList = []
    if 'txt' in fileName:
        lines = [line.strip() for line in open(file_dir1 + fileName)]
        for l in lines:
            lineWords = l.split()
            wordList.extend(lineWords)
        
        classification = classify_review(wordList, vocabulary)

        if classification == 0:
            #print("0")
            pos_review_count += 1
            # This is a positive review
        else:
            #print("1")
            neg_review_count += 1
            # This is a negative review
            
print("Number positive for POS test set (correct): ", pos_review_count)
print("Number negative for POS test set (incorrect): ", neg_review_count)

Number positive for POS test set (correct):  82
Number negative for POS test set (incorrect):  18


In [36]:
# For NEGATIVE test set

pos_review_count2 = 0
neg_review_count2 = 0

file_dir2 = 'test/neg/'
for fileName in os.listdir(file_dir2):
    wordList = []
    if 'txt' in fileName:
        lines = [line.strip() for line in open(file_dir2 + fileName)]
        for l in lines:
            lineWords = l.split()
            wordList.extend(lineWords)
        
    classification = classify_review(wordList, vocabulary)
    
    if classification == 0:
        pos_review_count2 += 1
        # This is a positive review
    else:
        neg_review_count2 += 1
        # This is a negative review
            
print("Number positive for NEG test set (incorrect): ", pos_review_count2)
print("Number negative for NEG test set (correct): ", neg_review_count2)

Number positive for NEG test set (incorrect):  15
Number negative for NEG test set (correct):  85
